#📌 Extracción

In [14]:
# Librarys used here

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import tabulate as tabulate
import folium
from pandas import json_normalize
from folium.plugins import HeatMap
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import json

#### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

In [11]:
with open("TelecomX_Data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df1= json_normalize(data) 
df1= pd.DataFrame(df1)
df1.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [15]:
# create a summary DataFrame with the required statistics
# including the number of nulls, unique values, max, min, mean, and standard
df1.replace("", np.nan, inplace=True)

summary = pd.DataFrame({
    'Columna': df1.columns,
    'Tipo de Dato': df1.dtypes.values,
    'Nulos': df1.isnull().sum().values,
    'Únicos': df1.nunique().values
})


summary = summary.sort_values(by='Nulos', ascending=False).reset_index(drop=True)

num_cols = df1.select_dtypes(include='number').columns

summary['Max'] = summary['Columna'].apply(lambda c: df1[c].max() if c in num_cols else None)
summary['Min'] = summary['Columna'].apply(lambda c: df1[c].min() if c in num_cols else None)
summary['Media'] = summary['Columna'].apply(lambda c: df1[c].mean() if c in num_cols else None)
summary['sd'] = summary['Columna'].apply(lambda c: df1[c].std() if c in num_cols else None)

print(summary.to_markdown(index=False))

| Columna                   | Tipo de Dato   |   Nulos |   Únicos |    Max |    Min |      Media |         sd |
|:--------------------------|:---------------|--------:|---------:|-------:|-------:|-----------:|-----------:|
| Churn                     | object         |     224 |        2 | nan    | nan    | nan        | nan        |
| customerID                | object         |       0 |     7267 | nan    | nan    | nan        | nan        |
| customer.gender           | object         |       0 |        2 | nan    | nan    | nan        | nan        |
| customer.SeniorCitizen    | int64          |       0 |        2 |   1    |   0    |   0.162653 |   0.369074 |
| customer.Partner          | object         |       0 |        2 | nan    | nan    | nan        | nan        |
| customer.Dependents       | object         |       0 |        2 | nan    | nan    | nan        | nan        |
| customer.tenure           | int64          |       0 |       73 |  72    |   0    |  32.3465   |  24.5

#🔧 Transformación

#📊 Carga y análisis

#📄Informe final